# Capstone Notebook

This notebook will be used for the Applied Data Science Specialization Capstone project on Coursera.

In [1]:
import pandas as pd
import numpy as np
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


# Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto

Week 3 Project

## Part 1: Scraping the data

In [2]:
#Use urllib, Beautifulsoup, and pandas to scrape the Wikipedia page. Steps outlined here --> 'https://simpleanalytical.com/how-to-web-scrape-wikipedia-python-urllib-beautiful-soup-pandas'

import urllib.request
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = urllib.request.urlopen(url)
from bs4 import BeautifulSoup
soup = BeautifulSoup(page, "lxml")

#Identify the datatable on the Wiki page
table=soup.find('table', class_='wikitable sortable')


In [3]:
#These steps pull the strings out of each row of the Wiki table and push them to a pandas dataframe

#3 empty lists for the data in each of the 3 columns
A = []
B = []
C = []

#This will extract the strings from each row of the table that has at least 3 items, excluding rows with unassigned boroughs
for row in table.findAll('tr'):
    cells = row.findAll('td')
    if len(cells) == 3 and len(cells[2].find(text = True)) > 1:
        A.append(cells[0].find(text = True))
        B.append(cells[1].find(text = True))
        C.append(cells[2].find(text = True))


#Assign the populated lists to a dataframe
df = pd.DataFrame(A, columns = ['PostalCode'])

df['Borough'] = B
df['Neighborhood'] = C

#This is a check to ensure no postal codes are listed more than once for multiple neighborhoods. There are 103 unique postal codes and 103 rows, so each is listed once.
print('Number of unique postal codes is', len(df['PostalCode'].unique()))
df.shape

Number of unique postal codes is 103


(103, 3)

## Part 2: Get coordinates

In [4]:
#Read the geospatial coordinates file into a pandas dataframe
co_df = pd.read_csv('/Users/collin/Downloads/Geospatial_Coordinates.csv')
co_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [8]:
#Sort the postal codes df by postal code to match the coordinates df for ease of transfering columns
df = df.sort_values('PostalCode')
df.reset_index(drop=True, inplace=True)
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.727929,-79.262029
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.794200,-79.262029
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.778517,-79.346556
3,M1G,Scarborough,Woburn,43.770120,-79.408493
4,M1H,Scarborough,Cedarbrae,43.745906,-79.352188


In [9]:
#Add the latitude and longitude columns to the postal code df
df['Latitude'] = co_df['Latitude']
df['Longitude'] = co_df['Longitude']
df.head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


## Part 3: Exploring and clustering the neighborhoods

In [30]:
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import json # library to handle JSON files

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [16]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="ca_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Toronto are 43.6534817, -79.3839347.


In [31]:
# create map of Canada
map_canada = folium.Map(location = [latitude, longitude], zoom_start = 10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_canada)  
    
map_canada

In [27]:
total_boroughs = len(df['Borough'].unique())
total_neighborhoods = len(df['Neighborhood'].unique())
print('The map above shows all', total_neighborhoods, 'groups of neighborhoods in all', total_boroughs, 'boroughs in Toronto')

The map above shows all 99 groups of neighborhoods in all 10 boroughs in Toronto


In [28]:
CLIENT_ID = '0CNXP4CYHXWC3SUR3X0JIAPZJ3TFRAZO2DJAOQZ1SMTIALJ0' # Foursquare ID
CLIENT_SECRET = 'QULEUCZUSLATDTC5DPKDVJD0VKJITJFA342FSVRDNDQL4R2E' # Foursquare Secret
VERSION = '20180605' # Foursquare API version

### Exploring Toronto neighborhoods

In [220]:
# Function to retrieve venues from the neighboorhoods

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    LIMIT = 100
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [221]:
# Use the above function on the neighborhoods dataframe to get all venues

toronto_venues = getNearbyVenues(names = df['Neighborhood'],
                                   latitudes = df['Latitude'],
                                   longitudes = df['Longitude']
                                  )

Malvern, Rouge

Rouge Hill, Port Union, Highland Creek

Guildwood, Morningside, West Hill

Woburn

Cedarbrae

Scarborough Village

Kennedy Park, Ionview, East Birchmount Park

Golden Mile, Clairlea, Oakridge

Cliffside, Cliffcrest, Scarborough Village West

Birch Cliff, Cliffside West

Dorset Park, Wexford Heights, Scarborough Town Centre

Wexford, Maryvale

Agincourt

Clarks Corners, Tam O'Shanter, Sullivan

Milliken, Agincourt North, Steeles East, L'Amoreaux East

Steeles West, L'Amoreaux West

Upper Rouge

Hillcrest Village

Fairview, Henry Farm, Oriole

Bayview Village

York Mills, Silver Hills

Willowdale, Newtonbrook

Willowdale, Willowdale East

York Mills West

Willowdale, Willowdale West

Parkwoods

Don Mills

Don Mills

Bathurst Manor, Wilson Heights, Downsview North

Northwood Park, York University

Downsview

Downsview

Downsview

Downsview

Victoria Village

Parkview Hill, Woodbine Gardens

Woodbine Heights

The Beaches

Leaside

Thorncliffe Park

East Toronto, Broadview N

In [222]:
# Size and preview of new dataframe with venues

print(toronto_venues.shape)
toronto_venues.head()

(2136, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Malvern, Rouge",43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
1,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,Scarborough Historical Society,43.788755,-79.162438,History Museum
3,"Guildwood, Morningside, West Hill",43.763573,-79.188711,RBC Royal Bank,43.766790,-79.191151,Bank
4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store


In [223]:
# Let's see how many venues were returned for each neighborhood

toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,4,4,4,4,4,4
"Alderwood, Long Branch",11,11,11,11,11,11
"Bathurst Manor, Wilson Heights, Downsview North",21,21,21,21,21,21
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",23,23,23,23,23,23
Berczy Park,58,58,58,58,58,58
"Birch Cliff, Cliffside West",4,4,4,4,4,4
"Brockton, Parkdale Village, Exhibition Place",23,23,23,23,23,23
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",17,17,17,17,17,17


In [96]:
# Let's see how many unique categories there are

print('There are {} unique categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 271 unique categories.


In [224]:
# Displaying the unique venue categories shows that there is a venue category 'Neighborhood' that we must rename

toronto_venues.loc[toronto_venues['Venue Category'] == 'Neighborhood']

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
319,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
447,Studio District,43.659526,-79.340923,Leslieville,43.662070,-79.337856,Neighborhood
1042,"Richmond, Adelaide, King",43.650571,-79.384568,Downtown Toronto,43.653232,-79.385296,Neighborhood
1124,"Harbourfront East, Union Station, Toronto Isla...",43.640816,-79.381752,Harbourfront,43.639526,-79.380688,Neighborhood


In [225]:
toronto_venues['Venue Category'] = toronto_venues['Venue Category'].replace({'Neighborhood': 'Area'})
toronto_venues.loc[toronto_venues['Venue Category'] == 'Area']

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
319,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Area
447,Studio District,43.659526,-79.340923,Leslieville,43.662070,-79.337856,Area
1042,"Richmond, Adelaide, King",43.650571,-79.384568,Downtown Toronto,43.653232,-79.385296,Area
1124,"Harbourfront East, Union Station, Toronto Isla...",43.640816,-79.381752,Harbourfront,43.639526,-79.380688,Area


In [226]:
# One hot encoding is useful for ranking venue categories by frequency for later clustering

toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood']

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot.reindex(columns =  fixed_columns)

toronto_onehot.head()

,Neighborhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Malvern, Rouge",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Rouge Hill, Port Union, Highland Creek",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Rouge Hill, Port Union, Highland Creek",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [227]:
# Let's see how big this new dataframe is

toronto_onehot.shape

(2136, 272)

In [228]:
# Now we group by neighborhood, take mean frequency for each venue category for later clustering

toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Agincourt,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000
1,"Alderwood, Long Branch",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.047619,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000
3,Bayview Village,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000
4,"Bedford Park, Lawrence Manor East",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.043478,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000
5,Berczy Park,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.017241,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000
6,"Birch Cliff, Cliffside West",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000
7,"Brockton, Parkdale Village, Exhibition Place",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000
8,"Business reply mail Processing Centre, South C...",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.058824
9,"CN Tower, King and Spadina, Railway Lands, Har...",0.0,0.000000,0.071429,0.071429,0.142857,0.142857,0.142857,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000


In [229]:
# Let's see how big this is

toronto_grouped.shape

(97, 272)

We're going to determine the top 5 most common venue types for each hood; here's a preview of the first

In [230]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood'].iloc[0:5]:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt
----
                       venue  freq
0                     Lounge  0.25
1             Breakfast Spot  0.25
2  Latin American Restaurant  0.25
3               Skating Rink  0.25
4          Accessories Store  0.00


----Alderwood, Long Branch
----
            venue  freq
0     Pizza Place  0.18
1        Pharmacy  0.09
2     Coffee Shop  0.09
3    Skating Rink  0.09
4  Sandwich Place  0.09


----Bathurst Manor, Wilson Heights, Downsview North
----
                       venue  freq
0                Coffee Shop  0.10
1                       Bank  0.10
2                       Park  0.05
3              Shopping Mall  0.05
4  Middle Eastern Restaurant  0.05


----Bayview Village
----
                 venue  freq
0                 Café  0.25
1                 Bank  0.25
2  Japanese Restaurant  0.25
3   Chinese Restaurant  0.25
4        Moving Target  0.00


----Bedford Park, Lawrence Manor East
----
                venue  freq
0         Coffee Shop  0.09
1    Sushi Restaurant 

Now let's get this into a dataframe

In [231]:
# First we need a function that sorts the venues in descending order

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the dataframe and view the top 10 venues for each neighborhood

In [232]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

#These neighborhoods were found to have no venues, so I dropped them:
#neighborhoods_venues_sorted.drop(index = 16, axis = 0, inplace = True)
#neighborhoods_venues_sorted.drop(index = 93, axis = 0, inplace = True)
    
neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Lounge,Latin American Restaurant,Breakfast Spot,Skating Rink,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run
1,"Alderwood, Long Branch",Pizza Place,Coffee Shop,Gym,Dance Studio,Skating Rink,Pharmacy,Pub,Athletics & Sports,Sandwich Place,Pool
2,"Bathurst Manor, Wilson Heights, Downsview North",Bank,Coffee Shop,Pharmacy,Supermarket,Middle Eastern Restaurant,Sushi Restaurant,Restaurant,Deli / Bodega,Fried Chicken Joint,Ice Cream Shop
3,Bayview Village,Japanese Restaurant,Café,Bank,Chinese Restaurant,Department Store,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Yoga Studio
4,"Bedford Park, Lawrence Manor East",Sushi Restaurant,Coffee Shop,Italian Restaurant,Sandwich Place,Restaurant,Liquor Store,Pharmacy,Indian Restaurant,Café,Pub


### Clustering the neighborhoods

Let's run k-means to cluster the neighborhoods into 5 clusters

In [233]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters = kclusters, random_state = 0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32)

Let's make a new dataframe that includes the cluster and top 10 venues for each neighborhood

In [234]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df

# merge toronto_grouped with toronto data (df) to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on = 'Neighborhood')

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,2.0,Fast Food Restaurant,Yoga Studio,Dog Run,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Doner Restaurant
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,1.0,Bar,History Museum,Yoga Studio,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Doner Restaurant
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,1.0,Electronics Store,Intersection,Mexican Restaurant,Breakfast Spot,Rental Car Location,Bank,Medical Center,Moving Target,Dim Sum Restaurant,Diner
3,M1G,Scarborough,Woburn,43.770992,-79.216917,1.0,Coffee Shop,Korean Restaurant,Yoga Studio,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Donut Shop
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,1.0,Fried Chicken Joint,Gas Station,Hakka Restaurant,Bakery,Bank,Athletics & Sports,Thai Restaurant,Caribbean Restaurant,Department Store,Dessert Shop


The cluster labels were converted to floats in the dataframe; we must make them integers.

Two neighborhood groups (Upper Rouge and Islington Avenue, Humber Valley Village) have 'nan' values for cluster labels, and errors were found upon closer inspection.

In [237]:
# Source of error not found, so let's drop these two neighborhoods. It is possible they have no listed venues.

toronto_merged.drop(index = 16, axis = 0, inplace = True)
toronto_merged.drop(index = 93, axis = 0, inplace = True)

In [241]:
# Change cluster labels to integers
toronto_merged['Cluster Labels'] = toronto_merged['Cluster Labels'].astype(int)
toronto_merged['Cluster Labels'].dtype

dtype('int64')

Finally, visualize the clusters!

In [242]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

The clustering result is a bit surprising and doesn't appear to show any pattern. But looking at the most common categories for cluster 4, for example, reveals high similarity.

In [243]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4.0]

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
91,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509,4,Baseball Field,Yoga Studio,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Falafel Restaurant
97,M9M,North York,"Humberlea, Emery",43.724766,-79.532242,4,Paper / Office Supplies Store,Baseball Field,Yoga Studio,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Doner Restaurant
